In [1]:
import numpy as np 
import pandas as pd 
import scipy.integrate

In [29]:

def load_constants():
    """Returns constants frequently used in this work"""
    params = {'vtl_max': 20,  # Max translation speed in AA/s
              'm_Rb': 7459,  # Proteinaceous mass of ribosome  in AA
              'Kd_cpc': 0.03,  # precursor dissociation constant in abundance units
              'Km': 5E-4,  # Nutrient monod constant in M
              'Y': 2.95E19,  # Yield coefficient
              'OD_conv': 1.5E17,  # Conversion factor from OD to AA mass.
              'Km_u': 3E-5,  # uncharged tRNA dissociation constant in abundance units
              'Kd_c': 3E-5,  # Charged tRNA dissociation constant in abundance units
              # Maximum tRNA synthesis rate  in abundance units per unit time
              'kappa_max': (64 * 5 * 3600) / 1E9,
              'tau': 1,  # ppGpp threshold parameter for charged/uncharged tRNA balance
              # Fraction of proteome deoveted to `other` proteins for E. coli.
              'phi_O': 0.55,
              }
    params['gamma_max'] = params['vtl_max'] * 3600 / params['m_Rb']
    return params

In [24]:
def unpack_params(params, args): 
    n_nuts = len(args['n_nutrients'])
    n_species = len(args(['n_species']))
    n_params = n_nuts + 4 # 4 is hard coded to account for M_O, M_Rb, tRNA_u, tRNA_c

    # Unpack the nutrient concentrations
    nutrients = [c for c in params[-n_nuts:]]
    masses = params[:-n_nuts]

    # Separate the params into species
    species = []
    for i in range(n_species):
        species.append(masses[i*n_params:n_params * (i + 1)])

    return nutrients, species




def compute_FPM(tRNA_u, tRNA_c, c_nt, args):
    n_nuts = len(c_nt)
    # Set ribosomal allocation, based on the ratio and enforcing positivity
    if tRNA_u <= 0:
        phi_Rb = (1 - args['phi_O'])
    else:
        ratio = tRNA_c/tRNA_u
        phi_Rb = (1 - args['phi_O']) * (ratio / (ratio + args['tau']))

    # Set the hierarchy and metabolic allocation 
    metabolic_suballocation = [] 
    if args['alpha_mode'] == 'dynamic':
        # We assume a monod function for the suballocation
        for i in range(n_nuts - 1):
            monod_fn = c_nt[i] / (c_nt[i] + args['alpha_Km'][i])
            if i == 0:
                metabolic_suballocation[i] = monod_fn
            else:
                metabolic_suballocation[i] = (1 - np.sum(metabolic_suballocation[:i]) * monod_fn)
        metabolic_suballocation[-1] = 1 - np.sum(metabolic_suballocation)

    elif args['alpha_mode'] == 'fixed':
        metabolic_suballocation = args['alpha']

    # Compute the rates
    gamma = args['gamma_max'] * tRNA_c / (tRNA_c + args['Km_c'])
    kappa = args['kappa_max'] * phi_Rb / (1 - args['phi_O'])
    nu = np.zeros_like(c_nt) 
    for i, c in enumerate(c_nt):
        nu[i] = args['nu_max'][i] * (tRNA_u / (tRNA_u + args['Km_u'])) * (c_nt[i] / (c_nt[i] + args['Km'][i])) 
 
    # Compute the net metabolic allocation
    phi_Mb = (1 - args['phi_O'] - phi_Rb) * metabolic_suballocation
    return phi_Rb, phi_Mb, nu, gamma, kappa

   
def compute_derivatives(masses, c_nt, args):
    """Computes the flux-parity system for a single community member""" 
    n_nuts = len(c_nt)
    M = np.sum(masses[:-2])

    # Unpack the metabolic masses
    M_Mb = masses[:n_nuts]

    # Unpack the ribosomal masse and charged, uncharged tRNA concentrations
    M_Rb, _,  tRNA_u, tRNA_c = masses[n_nuts:]   

    # Compute the flux-parity properties and unpack
    phi_Rb, phi_Mb, nu, gamma, kappa = compute_FPM(tRNA_u, tRNA_c, c_nt, args)

    # Compute the fluxes
    J_Rb = M_Rb * gamma / M
    J_Mb = np.sum(nu * M_Mb) / M

    # Compute the derivatives
    dM_Mb = phi_Mb * J_Rb
    dM_Rb = phi_Rb * J_Rb
    dM_O = args['phi_O'] * J_Rb
    dtRNA_u = kappa + J_Rb * (1 - tRNA_u) - J_Mb
    dtRNA_c = J_Mb - J_Rb * (1 + tRNA_c)
    dc_nt = -J_Mb * M/args['Y']
    return [dM_Mb, dM_Rb, dM_O, dtRNA_u, dtRNA_c], dc_nt


def ecosystem_dynamics(t, params, args):
    """Computes the dynamics of a community of self replicators"""
    ecosystem_args = args['ecosystem']
    community_args = args['community']
    nutrients, species = unpack_params(params, ecosystem_args)
    mass_derivatives = []
    nutrient_derivatives = np.zeros(len(nutrients))
    for i, s in enumerate(species):
        dM, dc = compute_derivatives(s, nutrients, community_args[i])
        for m in dM:
            mass_derivatives.append(m)
        for i, c in enumerate(dc):
            nutrient_derivatives[i] += append(c)
    du = np.array([np.array(mass_derivatives), nutrient_derivatives]).flatten()     
    return du




In [30]:
params = load_constants() 

# masses = np.array([0, 0, 1, 1, 1E-5, 1E-5])
# nutrients = np.array([1E-5, 1E-5, 1E-5])
# M_Rb, M_Mb, tRNA_u, tRNA_c = compute_derivatives(masses, nutrients, {})
# phi_Rb, phi_Mb = allocation(tRNA_u, tRNA_c, nutrients, {'phi_O':0.55, 'tau':1, 'alpha_mode':'fixed', 'alpha':np.array([0.6, 0.2, 0.2])})

In [31]:
params

{'vtl_max': 20,
 'm_Rb': 7459,
 'Kd_cpc': 0.03,
 'Km': 0.0005,
 'Y': 2.95e+19,
 'OD_conv': 1.5e+17,
 'Km_u': 3e-05,
 'Kd_c': 3e-05,
 'kappa_max': 0.001152,
 'tau': 1,
 'phi_O': 0.55,
 'gamma_max': 9.652768467623005}

In [28]:
phi_Mb

array([0.135, 0.045, 0.045])

In [15]:
tRNA_c

1e-05

In [16]:
M_Mb

[0, 0, 1]